In [1]:
# Load packages
import numpy as np
import pandas as pd
import warnings, gc, sys, json
import datetime as dt
from m5_utils import *

from itertools import product
from multiprocessing import Pool

#import tsfresh
from scipy.stats import kurtosis
from scipy.stats import skew

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn import preprocessing, metrics
from sklearn.decomposition import PCA, FastICA, TruncatedSVD
from sklearn.random_projection import GaussianRandomProjection, SparseRandomProjection
from sklearn.preprocessing import StandardScaler

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


pd.set_option('display.float_format', lambda x: '%.5f' % x)
pd.set_option('display.max_columns', 100)

In [2]:
with open('input/dict_m5.json') as json_file:
    dict_m5 = json.load(json_file)
with open('input/dict_m5_inv.json') as json_file:
    dict_m5_inv = json.load(json_file)

### Holidays feature

In [ ]:
calendar = pd.read_csv('input/calendar.csv')
calendar['date'] = pd.to_datetime(calendar['date'])

In [ ]:
holidays = pd.read_csv('input/holidays_consolidated.csv')
holidays['date'] = pd.to_datetime(holidays['Holiday_Date'])
del holidays['Holiday_Date']
holidays.shape

In [ ]:
holidays.drop_duplicates(['Holiday_Description','date'], keep='first', inplace= True)
holidays.drop(['state'], axis = 1, inplace = True)
holidays.shape

In [ ]:
# Merge holidays with calendar
calendar = calendar.merge(holidays, how = 'left', on = 'date')

In [ ]:
# There are four dates which werent covered by the calendar.
calendar[(calendar['Holiday_Description'].isna()==False)&(calendar['event_type_1'].isna()==True)]

In [ ]:
del weather['state']

### Weather feature

In [7]:
weather = pd.read_csv('input/weather_consolidated.csv')
print(weather.shape); weather.head()
weather = reduce_mem_usage(weather)
weather.dtypes

(6576, 9)


,date,totalSnow_cm,FeelsLikeC,HeatIndexC,WindChillC,humidity,precipMM,tempC,state_id
0,2011-01-01,0.30000,-1.40000,1.00000,-1.40000,49.60000,0.00000,1.00000,0
1,2011-01-10,0.00000,0.10000,2.50000,0.10000,55.00000,0.00000,2.50000,0
2,2011-01-11,0.00000,2.20000,3.60000,2.20000,43.10000,0.00000,3.60000,0
3,2011-01-12,0.00000,5.20000,5.90000,5.20000,48.30000,0.00000,5.90000,0
4,2011-01-13,0.00000,8.70000,9.70000,8.70000,60.70000,0.00000,9.70000,0


Mem. usage decreased to  0.14 Mb (68.0% reduction)


date             object
totalSnow_cm    float16
FeelsLikeC      float16
HeatIndexC      float16
WindChillC      float16
humidity        float16
precipMM        float16
tempC           float16
state_id           int8
dtype: object

In [ ]:
def add_datetime_features(df):
    
    df['date'] = pd.to_datetime(df['date'])
    df['weekend'] = (df['wday'] <3).astype(int)
    df['day'] = df['date'].dt.day
    df['quarter'] = df['date'].dt.quarter
    df['week'] = df['date'].dt.week
    df['dayofyear'] = df['date'].dt.dayofyear
    df['weekofyear'] = df['date'].dt.weekofyear
    
    return df
    # Add holidays 
    #add_holidays(df)

def add_holidays(df):
    
    #df['natal_2017'] = (pd.to_datetime('2017-12-25') - df['purchase_date']).dt.days.apply(lambda x: (1-x/37) if x > -7 and x < 30 else 0)
     print('no additional holidays so far')

def add_weather(df, weather):
    
    df = df.merge(weather, how='left', on=['date','state_id'])
    
    return df
    
def simple_fe(data):
    
    # demand features
    data['lag_t28'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28))
    data['lag_t29'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(29))
    data['lag_t30'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(30))
    data['lag_t31'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(31))
    
    data['rolling_mean_t7']   = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(7).mean())
    data['rolling_std_t7']    = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(7).std())
    
    data['rolling_mean_t30']  = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(30).mean())
    data['rolling_std_t30']   = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(30).std())
    
    data['rolling_mean_t90']  = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(90).mean())
    data['rolling_std_t90']   = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(90).std())
    
    data['rolling_mean_t180'] = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(180).mean())
    data['rolling_std_t180']  = data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(180).std())
    
    # price features
    data['lag_price_t1']           = data.groupby(['id'])['sell_price'].transform(lambda x: x.shift(1))
    data['price_change_t1']        = (data['lag_price_t1'] - data['sell_price']) / (data['lag_price_t1'])
    data['rolling_price_max_t365'] = data.groupby(['id'])['sell_price'].transform(lambda x: x.shift(1).rolling(365).max())
    data['price_change_t365']      = (data['rolling_price_max_t365'] - data['sell_price']) / (data['rolling_price_max_t365'])
    data.drop(['rolling_price_max_t365', 'lag_price_t1'], inplace = True, axis = 1)
    
    data['rolling_price_std_t7']   = data.groupby(['id'])['sell_price'].transform(lambda x: x.rolling(7).std())
    data['rolling_price_std_t30']  = data.groupby(['id'])['sell_price'].transform(lambda x: x.rolling(30).std())
    data['rolling_price_std_t90']  = data.groupby(['id'])['sell_price'].transform(lambda x: x.rolling(90).std())
    data['rolling_price_std_t180'] = data.groupby(['id'])['sell_price'].transform(lambda x: x.rolling(180).std())
    
    # time features
    add_datetime_features(data)
    
    return data

def _kurtosis(x):
    return kurtosis(x)


def CPT5(x):
    den = len(x)*np.exp(np.std(x))
    return sum(np.exp(x))/den

def skewness(x):
    return skew(x)

def SSC(x):
    x = np.array(x)
    x = np.append(x[-1], x)
    x = np.append(x,x[1])
    xn = x[1:len(x)-1]
    xn_i2 = x[2:len(x)]    # xn+1 
    xn_i1 = x[0:len(x)-2]  # xn-1
    ans = np.heaviside((xn-xn_i1)*(xn-xn_i2),0)
    return sum(ans[1:]) 

def wave_length(x):
    x = np.array(x)
    x = np.append(x[-1], x)
    x = np.append(x,x[1])
    xn = x[1:len(x)-1]
    xn_i2 = x[2:len(x)]    # xn+1 
    return sum(abs(xn_i2-xn))
    
def norm_entropy(x):
    tresh = 3
    return sum(np.power(abs(x),tresh))

def SRAV(x):    
    SRA = sum(np.sqrt(abs(x)))
    return np.power(SRA/len(x),2)

def mean_abs(x):
    return sum(abs(x))/len(x)

def zero_crossing(x):
    x = np.array(x)
    x = np.append(x[-1], x)
    x = np.append(x,x[1])
    xn = x[1:len(x)-1]
    xn_i2 = x[2:len(x)]    # xn+1
    return sum(np.heaviside(-xn*xn_i2,0))

# Funcao para criar alguns features estatisticas avancadas
def advanced_fe(data):
    
    df = pd.DataFrame()
    
    def mean_change_of_abs_change(x):
        return np.mean(np.diff(np.abs(np.diff(x))))
    
    for col in data.columns:
        
        print('col 1: ',col)
        if col in ['demand','sell_price']:
            
            df[col + '_mean'] = data.groupby(['id'])[col].mean()
            df[col + '_median'] = data.groupby(['id'])[col].median()
            df[col + '_max'] = data.groupby(['id'])[col].max()
            df[col + '_min'] = data.groupby(['id'])[col].min()
            df[col + '_std'] = data.groupby(['id'])[col].std()
            df[col + '_range'] = df[col + '_max'] - df[col + '_min']
            df[col + '_maxtoMin'] = df[col + '_max'] / df[col + '_min']
            df[col + '_mean_abs_chg'] = data.groupby(['id'])[col].apply(lambda x: np.mean(np.abs(np.diff(x))))
            df[col + '_mean_change_of_abs_change'] = data.groupby('id')[col].apply(mean_change_of_abs_change)
            df[col + '_abs_max'] = data.groupby(['id'])[col].apply(lambda x: np.max(np.abs(x)))
            df[col + '_abs_min'] = data.groupby(['id'])[col].apply(lambda x: np.min(np.abs(x)))
            df[col + '_abs_avg'] = (df[col + '_abs_min'] + df[col + '_abs_max'])/2
            
            print('col 2: ',col)
            
            # Advanced Features
            df[col + '_skew'] = data.groupby(['id'])[col].skew()
            df[col + '_mad'] = data.groupby(['id'])[col].mad()
            df[col + '_q25'] = data.groupby(['id'])[col].quantile(0.25)
            df[col + '_q75'] = data.groupby(['id'])[col].quantile(0.75)
            df[col + '_q95'] = data.groupby(['id'])[col].quantile(0.95)
            df[col + '_iqr'] = df[col + '_q75'] - df[col + '_q25']
            df[col + '_SSC'] = data.groupby(['id'])[col].apply(SSC) 
            df[col + '_skewness'] = data.groupby(['id'])[col].apply(skewness)
            df[col + '_wave_lenght'] = data.groupby(['id'])[col].apply(wave_length)
            df[col + '_norm_entropy'] = data.groupby(['id'])[col].apply(norm_entropy)
            df[col + '_SRAV'] = data.groupby(['id'])[col].apply(SRAV)
            df[col + '_kurtosis'] = data.groupby(['id'])[col].apply(_kurtosis) 
            df[col + '_zero_crossing'] = data.groupby(['id'])[col].apply(zero_crossing) 

    return df    


# Criando novas features atraces do PCA / ICA / GRP e SRP
def cluster_fe(data):
    
    n_comp = 4

    # tSVD
    tsvd = TruncatedSVD(n_components=n_comp, random_state=42)
    tsvd_results_df = tsvd.fit_transform(data[['item_id','dept_id','cat_id','store_id','state_id','wm_yr_wk','wday','month','year','event_name_1','event_type_1','event_name_2','event_type_2','snap_CA','snap_TX','snap_WI','sell_price','id']])

    # PCA
    pca = PCA(n_components=n_comp, random_state=42)
    pca2_results_df = pca.fit_transform(data[['item_id','dept_id','cat_id','store_id','state_id','wm_yr_wk','wday','month','year','event_name_1','event_type_1','event_name_2','event_type_2','snap_CA','snap_TX','snap_WI','sell_price','id']])

    # ICA
    ica = FastICA(n_components=n_comp, random_state=42)
    ica2_results_df = ica.fit_transform(data[['item_id','dept_id','cat_id','store_id','state_id','wm_yr_wk','wday','month','year','event_name_1','event_type_1','event_name_2','event_type_2','snap_CA','snap_TX','snap_WI','sell_price','id']])

    # GRP
    grp = GaussianRandomProjection(n_components=n_comp, eps=0.1, random_state=42)
    grp_results_df = grp.fit_transform(data[['item_id','dept_id','cat_id','store_id','state_id','wm_yr_wk','wday','month','year','event_name_1','event_type_1','event_name_2','event_type_2','snap_CA','snap_TX','snap_WI','sell_price','id']])

    # SRP
    srp = SparseRandomProjection(n_components=n_comp, dense_output=True, random_state=42)
    srp_results_df = srp.fit_transform(data[['item_id','dept_id','cat_id','store_id','state_id','wm_yr_wk','wday','month','year','event_name_1','event_type_1','event_name_2','event_type_2','snap_CA','snap_TX','snap_WI','sell_price','id']])


    # Append dos componentes com o dataset
    for i in range(1, n_comp+1):
        data['pca_demand'  + str(i)]  = pca2_results_df[:,i-1]
        data['ica_demand'  + str(i)]  = ica2_results_df[:,i-1]
        data['tsvd_demand' + str(i)]  = tsvd_results_df[:,i-1]
        data['grp_demand'  + str(i)]  = grp_results_df[:,i-1]
        data['srp_demand'  + str(i)]  = srp_results_df[:,i-1] 
        
    return data      
        

In [8]:
train = pd.read_pickle('input/test_v3.pkl')
print(train.shape); train.head()

(853720, 21)


,id,item_id,dept_id,cat_id,store_id,state_id,demand,source,date,wm_yr_wk,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,14370,1437,3,1,0,0,0,2,2016-04-25,11613,3,4,6,-1,-1,-1,-1,0,0,0,8.38281
1,14380,1438,3,1,0,0,0,2,2016-04-25,11613,3,4,6,-1,-1,-1,-1,0,0,0,3.97070
2,14390,1439,3,1,0,0,0,2,2016-04-25,11613,3,4,6,-1,-1,-1,-1,0,0,0,2.97070
3,14400,1440,3,1,0,0,0,2,2016-04-25,11613,3,4,6,-1,-1,-1,-1,0,0,0,4.64062
4,14410,1441,3,1,0,0,0,2,2016-04-25,11613,3,4,6,-1,-1,-1,-1,0,0,0,2.88086


In [11]:
train = add_weather(train, weather)
print(train.shape); train.head()

(853720, 28)


,id,item_id,dept_id,cat_id,store_id,state_id,demand,source,date,wm_yr_wk,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,totalSnow_cm,FeelsLikeC,HeatIndexC,WindChillC,humidity,precipMM,tempC
0,14370,1437,3,1,0,0,0,2,2016-04-25,11613,3,4,6,-1,-1,-1,-1,0,0,0,8.38281,0.00000,8.10156,11.70312,8.10156,31.50000,0.00000,11.70312
1,14380,1438,3,1,0,0,0,2,2016-04-25,11613,3,4,6,-1,-1,-1,-1,0,0,0,3.97070,0.00000,8.10156,11.70312,8.10156,31.50000,0.00000,11.70312
2,14390,1439,3,1,0,0,0,2,2016-04-25,11613,3,4,6,-1,-1,-1,-1,0,0,0,2.97070,0.00000,8.10156,11.70312,8.10156,31.50000,0.00000,11.70312
3,14400,1440,3,1,0,0,0,2,2016-04-25,11613,3,4,6,-1,-1,-1,-1,0,0,0,4.64062,0.00000,8.10156,11.70312,8.10156,31.50000,0.00000,11.70312
4,14410,1441,3,1,0,0,0,2,2016-04-25,11613,3,4,6,-1,-1,-1,-1,0,0,0,2.88086,0.00000,8.10156,11.70312,8.10156,31.50000,0.00000,11.70312


In [ ]:
train.fillna(0, inplace = True)

In [ ]:
%%time

# Chamando as funcoes de criacao de novas features de agrupamento
train = cluster_fe(train)

In [ ]:
%%time

# Chamando as funcoes de criacao de novas features
train = simple_fe(train)

In [ ]:
train['sell_price'] = train['sell_price'].astype(np.float32)

In [ ]:
%%time


#train = reduce_mem_usage(train)
# Chamando as funcoes de criacao de novas features estatisticas avancadas
adv_fe = advanced_fe(train)
adv_fe.reset_index(drop=False, inplace=False)

# Realizando o merge final
df_merge = pd.merge(train, adv_fe, on='id')